<a href="https://colab.research.google.com/github/mik-nn/Sensor_Classification/blob/main/100__Acc_Count_Dif_Threshold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
#@title Грузим библиотеки
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import collections
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from imblearn.over_sampling import SMOTE, ADASYN,BorderlineSMOTE,SVMSMOTE,KMeansSMOTE
from imblearn.metrics import classification_report_imbalanced

In [58]:
#@title Загрузка данных
data = pd.read_csv('100_acc.csv')
data.drop('signal_id', axis=1,inplace=True)
# Смотрим сколько меток для класса 1 и для класса 2 чтоы понимать степень разбалансированности
data_pos = data[data.label == 1]
data_neg = data[data.label == 0]

print(f'Класс 1:{data_pos.shape}, Класс 0:{data_neg.shape}')


Класс 1:(51, 129), Класс 0:(603, 129)


In [59]:
def count_threshold(series,threshold = .051):
  count = 0
  for val in series:
    if val > threshold: count+=1
  return count

df_dif = data.copy().drop('label', axis=1)
df_dif = df_dif.iloc[:,:-1]
#df_dif['Mean'] = df_dif.mean(axis=1)
#for col in df_dif.columns:
#    df_dif[col] = df_dif[col] -df_dif['Mean']
df_dif = df_dif.diff(axis=1).abs()
df_dif['Mean'] = df_dif.mean(axis=1)
df_dif['threshold'] = df_dif.apply(count_threshold,axis=1)

fig = px.scatter(df_dif, x='Mean', y = 'threshold', color=data.label)
fig.show()


In [60]:
data['Mean']=df_dif.Mean
data['threshold']= df_dif.threshold
data =data.iloc[:,-3:]

In [61]:
data

,label,Mean,threshold
0,0,0.073548,58
1,0,0.104062,79
2,0,0.072856,62
3,0,0.086051,69
4,0,0.065267,59
...,...,...,...
649,0,0.103357,80
650,0,0.039445,31
651,0,0.067981,59
652,0,0.103927,86


In [62]:
#data =data.drop(420,axis=0)

In [63]:
data_neg = data[data.label ==0]
data_pos = data[data.label == 1]
X = data.iloc[:,1:].to_numpy()
Y = data.iloc[:,0].to_numpy()
X_pos = data_pos.iloc[:,1:].to_numpy()
Y_pos = data_pos.iloc[:,0].to_numpy()
X_neg = data_neg.iloc[:,1:].to_numpy()
Y_neg = data_neg.iloc[:,0].to_numpy()

In [64]:

for rs in  range(50):
  kfold = StratifiedKFold(n_splits=4, shuffle=True, random_state=rs)

  # Подсчет осуществляемой точности
  score = []
  scaler = StandardScaler()
  #for train_index, test_index in kfold.split(X, Y):
  for (train_pos_index, test_pos_index),(train_neg_index,test_neg_index) \
                    in zip(kfold.split(X_pos, Y_pos),kfold.split(X_neg,Y_neg)):
    #  print(train_pos_index, test_pos_index,train_neg_index,test_neg_index)
      X_train = np.concatenate((X_pos[train_pos_index], X_neg[train_neg_index]))
      Y_train = np.concatenate((Y_pos[train_pos_index], Y_neg[train_neg_index]))
      X_test = np.concatenate((X_pos[test_pos_index], X_neg[test_neg_index]))
      Y_test = np.concatenate((Y_pos[test_pos_index], Y_neg[test_neg_index]))
      X_train = scaler.fit_transform(X_train)
      X_test = scaler.transform(X_test)
  #    X_train, X_test = X[train_index], X[test_index]
  #    Y_train, Y_test = Y[train_index], Y[test_index]
  #    X_resampled, y_resampled = SVMSMOTE().fit_resample(X_train, Y_train)
  #    clf = LogisticRegression().fit(X_train, Y_train)
      clf = DecisionTreeClassifier().fit(X_train, Y_train)
  #    clf = DecisionTreeClassifier().fit(X_resampled, y_resampled)
      predictions = clf.predict(X_test)
      #(predictions)
      score.append(accuracy_score(Y_test, predictions))
#      cri = classification_report_imbalanced(Y_test, predictions)
#      print('Accuracy: ', accuracy)
      #print(f'{cri}')
  if np.array(score).mean() == 1: break
  print(f'{rs} Аккуратность: {np.array(score).mean():.3f}')



0 Аккуратность: 0.994
1 Аккуратность: 0.994
2 Аккуратность: 0.992
3 Аккуратность: 0.994
4 Аккуратность: 0.995
5 Аккуратность: 0.995
6 Аккуратность: 0.994
7 Аккуратность: 0.991
8 Аккуратность: 0.994
9 Аккуратность: 0.994
10 Аккуратность: 0.994
11 Аккуратность: 0.992
12 Аккуратность: 0.994
13 Аккуратность: 0.995
14 Аккуратность: 0.995
15 Аккуратность: 0.992
16 Аккуратность: 0.995
17 Аккуратность: 0.994
18 Аккуратность: 0.994
19 Аккуратность: 0.995
20 Аккуратность: 0.995
21 Аккуратность: 0.995
22 Аккуратность: 0.991
23 Аккуратность: 0.994
24 Аккуратность: 0.988
25 Аккуратность: 0.994
26 Аккуратность: 0.994
27 Аккуратность: 0.992
28 Аккуратность: 0.994
29 Аккуратность: 0.992
30 Аккуратность: 0.995
31 Аккуратность: 0.995
32 Аккуратность: 0.995
33 Аккуратность: 0.995
34 Аккуратность: 0.995
35 Аккуратность: 0.992
36 Аккуратность: 0.991
37 Аккуратность: 0.994
38 Аккуратность: 0.989
39 Аккуратность: 0.994
40 Аккуратность: 0.995
41 Аккуратность: 0.992
42 Аккуратность: 0.991
43 Аккуратность: 0.99

In [65]:
  kfold = StratifiedKFold(n_splits=4, shuffle=True, random_state=41)

  # Подсчет осуществляемой точности
  score = []
  scaler = StandardScaler()
  #for train_index, test_index in kfold.split(X, Y):
  for (train_pos_index, test_pos_index),(train_neg_index,test_neg_index) \
                    in zip(kfold.split(X_pos, Y_pos),kfold.split(X_neg,Y_neg)):
    #  print(train_pos_index, test_pos_index,train_neg_index,test_neg_index)
      X_train = np.concatenate((X_pos[train_pos_index], X_neg[train_neg_index]))
      Y_train = np.concatenate((Y_pos[train_pos_index], Y_neg[train_neg_index]))
      X_test = np.concatenate((X_pos[test_pos_index], X_neg[test_neg_index]))
      Y_test = np.concatenate((Y_pos[test_pos_index], Y_neg[test_neg_index]))

  #    X_train, X_test = X[train_index], X[test_index]
  #    Y_train, Y_test = Y[train_index], Y[test_index]
  #    X_resampled, y_resampled = SVMSMOTE().fit_resample(X_train, Y_train)
  #    clf = LogisticRegression().fit(X_train, Y_train)
      clf = DecisionTreeClassifier().fit(X_train, Y_train)
  #    clf = DecisionTreeClassifier().fit(X_resampled, y_resampled)
      predictions = clf.predict(X_test)
      #(predictions)
      score.append(accuracy_score(Y_test, predictions))
print(np.array(score).mean(),score)

0.992359229147847 [0.9817073170731707, 1.0, 0.9939024390243902, 0.9938271604938271]


In [66]:
!pip -q install catboost

In [67]:
from catboost import CatBoostClassifier
kfold = StratifiedKFold(n_splits=4, shuffle=True, random_state=999)

# Подсчет осуществляемой точности
score = []
#scaler = StandardScaler()
#for train_index, test_index in kfold.split(X, Y):
for (train_pos_index, test_pos_index),(train_neg_index,test_neg_index) \
                    in zip(kfold.split(X_pos, Y_pos),kfold.split(X_neg,Y_neg)):
    #  print(train_pos_index, test_pos_index,train_neg_index,test_neg_index)
      X_train = np.concatenate((X_pos[train_pos_index], X_neg[train_neg_index]))
      Y_train = np.concatenate((Y_pos[train_pos_index], Y_neg[train_neg_index]))
      X_test = np.concatenate((X_pos[test_pos_index], X_neg[test_neg_index]))
      Y_test = np.concatenate((Y_pos[test_pos_index], Y_neg[test_neg_index]))
#      X_train = scaler.fit_transform(X_train)
#      X_test = scaler.transform(X_test)
  #    X_train, X_test = X[train_index], X[test_index]
  #    Y_train, Y_test = Y[train_index], Y[test_index]
  #    X_resampled, y_resampled = SVMSMOTE().fit_resample(X_train, Y_train)
  #    clf = LogisticRegression().fit(X_train, Y_train)
      clf = CatBoostClassifier().fit(X_train, Y_train,verbose=0)
  #    clf = DecisionTreeClassifier().fit(X_resampled, y_resampled)
      predictions = clf.predict(X_test)
      #(predictions)
#      if accuracy_score(Y_test, predictions) < 1:
 #       print(test_neg_index,X_neg[test_neg_index],accuracy_score(Y_test, predictions))
      score.append(accuracy_score(Y_test, predictions))
print(np.array(score).mean(),score)

0.9953891900030112 [1.0, 0.9939024390243902, 1.0, 0.9876543209876543]


In [69]:
  from sklearn.cluster import KMeans
  kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=46)

  # Подсчет осуществляемой точности
  score = []
  #scaler = StandardScaler()
  #for train_index, test_index in kfold.split(X, Y):
  for (train_pos_index, test_pos_index),(train_neg_index,test_neg_index) \
                    in zip(kfold.split(X_pos, Y_pos),kfold.split(X_neg,Y_neg)):
    #  print(train_pos_index, test_pos_index,train_neg_index,test_neg_index)
      X_train = np.concatenate((X_pos[train_pos_index], X_neg[train_neg_index]))
      Y_train = np.concatenate((Y_pos[train_pos_index], Y_neg[train_neg_index]))
      X_test = np.concatenate((X_pos[test_pos_index], X_neg[test_neg_index]))
      Y_test = np.concatenate((Y_pos[test_pos_index], Y_neg[test_neg_index]))

  #    X_train, X_test = X[train_index], X[test_index]
  #    Y_train, Y_test = Y[train_index], Y[test_index]
  #    X_resampled, y_resampled = SVMSMOTE().fit_resample(X_train, Y_train)
  #    clf = LogisticRegression().fit(X_train, Y_train)
      clf = KMeans(n_clusters=2,).fit(X_train, Y_train)
  #    clf = DecisionTreeClassifier().fit(X_resampled, y_resampled)
      predictions = clf.predict(X_test)
#      print(predictions)
      #(predictions)
      score.append(accuracy_score(Y_test, predictions))
  print(np.array(score).mean(),score)

0.4691971387391235 [0.5681818181818182, 0.4580152671755725, 0.42748091603053434, 0.43846153846153846, 0.45384615384615384]
